<a href="https://colab.research.google.com/github/madch3m/tversky-similarity-grad/blob/visualization/Tversky_Shared_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tversky Layer GPT Implementation

In [1]:
!pip install transformers torch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions
class TverskyProjectionLayer(nn.Module):
    def __init__(self, in_features, out_features, alpha=0.5, beta=0.5,gamma=1.0):
        super(TverskyProjectionLayer, self).__init__()
        self.features = nn.Parameter(torch.randn(out_features,in_features))
        self.prototypes = nn.Parameter(torch.randn(out_features, in_features))
        self.beta = nn.Parameter(torch.randn(in_features, in_features))
        self.gamma = nn.Parameter(torch.tensor(alpha))

        nn.init.xavier_uniform_(self.prototypes)
        nn.init.xavier_uniform_(self.features)

    def commonality(self, x, prototype):
        x_feature_activations = torch.matmul(x, self.features)
        x_features = F.relu(x_feature_activations)
        prototype_feature_activations = torch.matmul(prototype, self.features)
        prototype_features = F.relu(prototype_feature_activations)

        common_features = torch.sum(torch.min(x_features,prototype_features))

        return common, x_features, prototype_features

    def distinct_features(self, features_alpha ,features_beta):
        distinctive = torch.sum(F.relu(features_alpha - features_beta),dim=-1)
        return distinctive

    def tversky_similarity(self, x, prototype):
        common, x_features, prototype_features = self.commonality(x,prototype)
        distinctive_x = self.distinctive(x_features, prototype_features)
        distinctive_prototype = self.distinctive(prototype_features,x_features)

        numerator = self.gamma * common
        denominator = (self.gamma * common + torch.abs(self.alpha) * distinctive_x + torch.abs(self.beta) * distinctive_prototype + 1e-8)
        similarity = numerator / denominator

        return similarity

    def forward(self, x):
        input_shape = x.shape

        if len(x.shape) == 3:
            batch_size, seq_len, in_features = x.shape
            x = x.view(-1, in_features)

        else:
            batch_size = None
            seq_len = None

        similarities = []

        for i in range(self.prototypes.size(0)):
            similarity = self.tversky_similarity(x,self.prototypes[i])
            similarities.append(similarity)

        output = torch.stack(similarities, dim=0)

        if batch_size is not None and seq_len is not None:
            output = output.view(batch_size, seq_len, -1)
        return output

##TverskyHeadModel

In [2]:
class TverskyHead(nn.Module):
    def __init__(self, config, alpha=0.5, beta=0.5, gamma=1.0):
        super(TverskyHead, self).__init__()

        self.config = config
        self.transformer = GPT2Model(config)

        self.tversky_head = TverskyProjectionLayer(config.n_embd, config.vocab_size, alpha=alpha, beta=beta, gamma=gamma)

    def forward(self,input_ids=None, attention_mask=None, labels=None, **kwargs):
        transformer_outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        hidden_layers = transformer_outputs.last_hidden_state

        lm_logits = self.tversky_head(hidden_layers)
        loss = None
        if labels is not None:
            shift_logits = lm_logits[...,:-1,:].contiguous()
            shift_labels = labels[...,1:].contiguous()

            loss_f = nn.CrossEntropyLoss()
            loss = loss_fc(shift_logits.view(-1, shift_logits.size(-1)),
                           shift_labels.view(-1))

            return CausalLMOutputWithCrossAttentions(loss=loss, logits=lm_logits, hidden_states=transformer_outputs.hidden_states,
                                                     attentions=transformer_outputs.attentions)
    def generate(self, input_ids, max_length=50, temperature=1.0, **kwargs):
        self.eval()
        with torch.no_grad():
            for _ in range(max_length - input_ids.size(1)):
                outputs = self.forward(input_ids)
                next_token_logits = outputs.logits[:,-1,:] / temperature
                next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
                input_ids = torch.cat([input_ids, next_token], dim=1)

                if next_token.item() == self.config.eos_token_id:
                    break

        return input_ids

In [6]:
def main():
  print("GPT-2 with Tversky Head Example")
  print("=" * 70)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")
  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  tokenizer.pad_token = tokenizer.eos_token

  config = GPT2Config.from_pretrained("gpt2")

  print("Initializing GPT-2 with Tversky head...")

  model = TverskyHead(config, alpha=0.5,beta=0.5,gamma=1.0).to(device)

  total_params = sum(p.numel() for p in model.parameters())
  tversky_params = sum(p.numel() for p in model.tversky_head.parameters())
  print(f"Total parameters: {total_params}")
  print(f"Tversky parameters: {tversky_params}")

In [7]:
main()

GPT-2 with Tversky Head Example
Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Initializing GPT-2 with Tversky head...
Total parameters: 202224385
Tversky parameters: 77784577


In [3]:
class TverskyLinear(nn.Module):
    def __init_(self, in_features, out_features, alpha=0.5,beta=0.5, bias=True,fixed_params=True):
        super(TverskyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.prototypes = nn.Parameter(torch.randn(out_features, in_features))

        self.features = nn.Parameter(torch.randn(in_features, in_features))

        if fixed_params:
            self.alpha = nn.register_buffer(torch.tensor(alpha))
            self.beta = nn.register_buffer(torch.tensor(beta))
            self.gamma = nn.register_buffer(torch.tensor(gamma))
        else:
            self.alpha = nn.Parameter('alpha',torch.tensor(alpha))
            self.beta = nn.Parameter('beta', torch.tensor(beta))
            self.gamma = nn.Parameter('gamma', torch.tensor(gamma))

        if bias:
            self.bias = nn.Parameter(torch.randn(out_features))
        else:
            self._register_parameter('bias',None)

    def _reset_parameters(self):
        nn.init.xavier_uniform_(self.prototypes)
        nn.init.xavier_uniform_(self.features)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def commonality(self, x, prototype):
        x_feature_activations = torch.matmul(x, self.features)
        x_features = F.relu(x_feature_activations)
        prototype_feature_activations = torch.matmul(prototype, self.features)
        prototype_features = F.relu(prototype_feature_activations)

        common_features = torch.sum(torch.min(x_features,prototype_features))

        return common, x_features, prototype_features

    def distinct_features(self, features_alpha ,features_beta):
        distinctive = torch.sum(F.relu(features_alpha - features_beta),dim=-1)
        return distinctive

    def tversky_similarity(self, x, prototype):
        common, x_features, prototype_features = self.commonality(x,prototype)
        distinctive_x = self.distinctive(x_features, prototype_features)
        distinctive_prototype = self.distinctive(prototype_features,x_features)

        numerator = self.gamma * common
        denominator = (self.gamma * common + torch.abs(self.alpha) * distinctive_x + torch.abs(self.beta) * distinctive_prototype + 1e-8)
        similarity = numerator / denominator

        return similarity


Minimize the number of parameters by replacing the QKV layers with Tversky linear layers

In [4]:
class TverskyAttention(nn.Module):
    def __init__(self,embed_dim,num_heads, dropout=0.1, bias=True,alpha=0.5,beta=0.5,gamma=1.0):
        super(TverskyAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_proj = TverskyProjectionLayer(embed_dim, embed_dim, alpha=alpha, beta=beta, gamma=gamma)

In [5]:
class GlobalFeature:
    _instance = None
    _feature_matrices = {}

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(GlobalFeature, cls).__new__(cls)
        return cls._instance
    def register_feature(self,key, feature_matrix):
        self._feature_matrices[key] = feature_matrix

    def get_feature(self, key):
        return self._feature_matrices.get(key)

    def clear(self):
        self._feature_matrices.clear()

    def has_key(self, key):
        return key in self._feature_matrices

In [29]:
class SharedTverskyLinear(nn.Module):
    def __init__(self, in_features, out_features, feature_key='main', alpha=0.5, beta=0.5, gamma=1.0, bias=True, share_features=True):
        super(SharedTverskyLinear, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        self.feature_key = feature_key
        self.share_feature_params = share_features

        self.prototypes = nn.Parameter(torch.randn(out_features, in_features))

        self.registry = GlobalFeature()
        feature_matrix_key = f"{feature_key}_{in_features}"

        if not self.registry.has_key(feature_matrix_key):
            features = nn.Parameter(torch.randn(in_features, in_features))
            self.registry.register_feature(feature_matrix_key, features)

        self._feature_matrix_key = feature_matrix_key

        if share_features:
            param_key = f"tversky_params_{feature_key}" # Changed key to avoid conflict
            if not self.registry.has_key(param_key):
                params = {
                    'alpha': nn.Parameter(torch.tensor(alpha)),
                    'beta': nn.Parameter(torch.tensor(beta)),
                    'gamma': nn.Parameter(torch.tensor(gamma))
                }
                self.registry.register_feature(param_key, params)
            self._param_key = param_key
            self._alpha = None
            self._beta = None
            self._gamma = None
        else:
            self._alpha = nn.Parameter(torch.tensor(alpha))
            self._beta = nn.Parameter(torch.tensor(beta))
            self._gamma = nn.Parameter(torch.tensor(gamma))
            self._param_key = None # Initialize shared parameter key to None


        if bias:
            self.bias = nn.Parameter(torch.randn(out_features))
        else:
            self.register_parameter('bias', None)

        self._reset_parameters()

    @property
    def features(self):
        return self.registry.get_feature(self._feature_matrix_key)

    @property
    def alpha(self):
        if self._param_key:
            return self.registry.get_feature(self._param_key)['alpha']
        return self._alpha

    @alpha.setter
    def alpha(self, value):
        if not self._param_key:
           self._alpha = value

    @property
    def beta(self):
        if self._param_key:
            return self.registry.get_feature(self._param_key)['beta']
        return self._beta

    @beta.setter
    def beta(self, value):
        if not self._param_key:
            self._beta = value

    @property
    def gamma(self):
        if self._param_key:
            return self.registry.get_feature(self._param_key)['gamma']
        return self._gamma

    @gamma.setter
    def gamma(self, value):
        if not self._param_key:
            self._gamma = value

    def _reset_parameters(self):
        nn.init.xavier_uniform_(self.prototypes)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def tversky_similarity_batch(self, x, prototypes):
        batch_size = x.size(0)
        features = self.features
        x_activations = torch.matmul(x, features)
        x_features = F.relu(x_activations)

        prototypes_activations = torch.matmul(prototypes, features)
        prototype_features = F.relu(prototypes_activations)

        x_features_expanded = x_features.unsqueeze(1)
        prototype_features_expanded = prototype_features.unsqueeze(0)

        common_features = torch.sum(torch.min(x_features_expanded, prototype_features_expanded),dim=-1)
        distinctive_x = torch.sum(F.relu(x_features_expanded - prototype_features_expanded), dim=-1)
        distinctive_prototypes = torch.sum(F.relu(prototype_features_expanded - x_features_expanded), dim=-1)


        numerator = self.gamma * common_features
        denominator = (self.gamma * common_features + torch.abs(self.alpha) * distinctive_x + torch.abs(self.beta) * distinctive_prototypes + 1e-8)
        similarity = numerator / denominator
        return similarity

    def forward(self, x):
        original_shape = x.shape[:-1]
        x_flattened = x.view(-1, self.in_features)
        output = self.tversky_similarity_batch(x_flattened, self.prototypes)

        if self.bias is not None:
            output += self.bias

        output = output.view(*original_shape, self.out_features)

        return output

In [31]:
class TverskyAttentionShared(nn.Module):
    def __init__(self, embed_dim, num_heads, feature_key='main', dropout=0.1, bias=True, alpha=0.5, beta=0.5, gamma=1.0):
        super(TverskyAttentionShared, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_proj = SharedTverskyLinear(embed_dim, embed_dim, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma, bias=bias)
        self.k_proj = SharedTverskyLinear(embed_dim, embed_dim, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma,bias=bias)
        self.v_proj = SharedTverskyLinear(embed_dim, embed_dim, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma, bias=bias)

        self.out_proj = SharedTverskyLinear(embed_dim, embed_dim, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma, bias=bias)

        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

    def __split_heads(self, tensor, batch_size):
        return tensor.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1,2)
    def _merge_heads(self, tensor, batch_size):
        return tensor.transpose(1,2).contiguous().view(batch_size,-1, self.embed_dim)

    def forward(self, hidden_states, attention_mask=None, layer_past=None, use_cache=False,output_attentions=False):
        batch_size, seq_length = hidden_states.size()[:2]
        query = self.q_proj(hidden_states)
        key = self.k_proj(hidden_states)
        value = self.v_proj(hidden_states)

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat([past_key, key], dim=-2)
            value = torch.cat([past_value, value],dim=-2)
        present = (key, value) if use_cache else None

        attn_weights = torch.matmul(query, key.transpose(-1,-2))
        attn_weights = attn_weights / (self.head_dim**0.5)

        if attention_mask is not None:
            attn_weights = F.softmax(attn_weights, dim=-1)

        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.attn_dropout(attn_weights)

        attn_output = torch.matmul(attn_weights, value)
        attn_output = self._merge_heads(attn_output, batch_size)
        attn_output = self.out_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            output += (attn_weights,)
        return outputs

In [20]:
class SharedTverskyNetwork(nn.Module):
    def __init__(self, embed_dim, intermediate_dim, feature_key='main', dropout=0.1, alpha=0.5,beta=0.5, gamma=1.0):
        super(SharedTverskyNetwork, self).__init__()
        self.layer1 = SharedTverskyLinear(embed_dim,intermediate_dim, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma)
        self.layer2 = SharedTverskyLinear(intermediate_dim, embed_dim, feature_key=f"{feature_key}_intermediate", alpha=alpha, beta=beta, gamma=gamma)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, hidden_states):
        hidden_states = self.layer1(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.layer2(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states


In [25]:
class TverskyTransformerBlock(nn.Module):
    def __init__(self, config, feature_key='main', alpha=0.5, beta=0.5, gamma=1.0):
        super(TverskyTransformerBlock, self).__init__()
        hidden_size = config.n_embd
        inner_dim = config.n_inner if config.n_inner is not None else 4 * hidden_size

        self.ln_1 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        self.attn = TverskyAttentionShared(embed_dim=hidden_size, num_heads=config.n_head, feature_key=feature_key, dropout=config.attn_pdrop, alpha=alpha, beta=beta, gamma=gamma)

        self.ln_2 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        self.network = SharedTverskyNetwork(embed_dim=hidden_size, intermediate_dim=inner_dim,feature_key=feature_key, dropout=config.resid_pdrop, alpha=alpha, beta=beta, gamma=gamma)


    def forward(self, hidden_states, attention_mask=None, layer_past=None, use_cache=False, output_attentions=False):

        residual = hidden_states
        attn_outputs = self.attn(hidden_states, attention_mask=attention_mask, layer_past=layer_past, use_cache=use_cache)

        attn_output = attn_outputs[0]
        outputs = attn_outputs[1:]

        hidden_states = residual + attn_output

        residual = hidden_states
        hidden_states = self.ln_2(hidden_states)
        ffn_states = self.network(hidden_states)
        hidden_states = residual + ffn_states

        if use_cache:
            outputs = (hidden_states,) + outputs
        else:
            outputs = (hidden_states,) + outputs[1:]
        return outputs

In [23]:
class GPT2TverskyModel(nn.Module):
    """
    GPT-2 model with globally shared Tversky feature matrices.
    """
    def __init__(self, config, feature_key='main', alpha=0.5, beta=0.5, gamma=1.0):
        super(GPT2TverskyModel, self).__init__()

        self.config = config
        self.embed_dim = config.n_embd

        # Clear registry for fresh start
        GlobalFeature().clear()

        print(f"\n{'='*70}")
        print(f"Building GPT-2 with Shared Tversky Layers")
        print(f"{'='*70}")

        # Embeddings (standard)
        self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
        self.wpe = nn.Embedding(config.n_positions, self.embed_dim)
        self.drop = nn.Dropout(config.embd_pdrop)

        # Transformer blocks with shared features
        self.h = nn.ModuleList([
            TverskyTransformerBlock(config, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma)
            for _ in range(config.n_layer)
        ])

        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, SharedTverskyLinear):
            module.prototypes.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self,input_ids=None,attention_mask=None,position_ids=None,past_key_values=None,use_cache=None,output_attentions=None,output_hidden_states=None):
        use_cache = use_cache if use_cache is not None else self.config.use_cache

        batch_size, seq_length = input_ids.size()

        if position_ids is None:
            past_length = past_key_values[0][0].size(-2) if past_key_values is not None else 0
            position_ids = torch.arange(
                past_length, seq_length + past_length,
                dtype=torch.long, device=input_ids.device
            )
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds
        hidden_states = self.drop(hidden_states)

        if attention_mask is not None:
            attention_mask = attention_mask.view(batch_size, -1)
            attention_mask = attention_mask[:, None, None, :]
            attention_mask = attention_mask.to(dtype=hidden_states.dtype)
            attention_mask = (1.0 - attention_mask) * torch.finfo(hidden_states.dtype).min

        presents = () if use_cache else None
        all_hidden_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None

        for i, block in enumerate(self.h):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_past = past_key_values[i] if past_key_values is not None else None

            outputs = block(
                hidden_states,
                attention_mask=attention_mask,
                layer_past=layer_past,
                use_cache=use_cache,
                output_attentions=output_attentions
            )

            hidden_states = outputs[0]

            if use_cache:
                presents = presents + (outputs[1],)

            if output_attentions:
                all_attentions = all_attentions + (outputs[2 if use_cache else 1],)

        hidden_states = self.ln_f(hidden_states)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        return {
            'last_hidden_state': hidden_states,
            'past_key_values': presents,
            'hidden_states': all_hidden_states,
            'attentions': all_attentions,
        }


class GPT2SharedTverskyLMHead(nn.Module):
    """
    GPT-2 LM with shared Tversky layers throughout.
    """
    def __init__(self, config, feature_key='main', alpha=0.5, beta=0.5, gamma=1.0):
        super(GPT2SharedTverskyLMHead, self).__init__()

        self.config = config
        self.transformer = GPT2TverskyModel(
            config, feature_key=feature_key, alpha=alpha, beta=beta, gamma=gamma
        )

        # LM head with shared features
        self.lm_head = SharedTverskyLinear(
            config.n_embd,
            config.vocab_size,
            feature_key=feature_key,
            alpha=alpha, beta=beta, gamma=gamma,
            bias=False
        )

        print(f"{'='*70}\n")

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        transformer_outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )

        hidden_states = transformer_outputs['last_hidden_state']
        lm_logits = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        return CausalLMOutputWithCrossAttentions(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.get('past_key_values'),
            hidden_states=transformer_outputs.get('hidden_states'),
            attentions=transformer_outputs.get('attentions'),
        )

    def generate(self, input_ids, max_length=50, temperature=1.0, top_k=50):
        self.eval()
        with torch.no_grad():
            for _ in range(max_length - input_ids.size(1)):
                outputs = self.forward(input_ids)
                next_token_logits = outputs.logits[:, -1, :] / temperature

                if top_k > 0:
                    indices_to_remove = next_token_logits < torch.topk(next_token_logits, top_k)[0][..., -1, None]
                    next_token_logits[indices_to_remove] = float('-inf')

                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                input_ids = torch.cat([input_ids, next_token], dim=1)

                if next_token.item() == self.config.eos_token_id:
                    break

        return input_ids

In [32]:
import torch.nn as nn
def count_parameters(model):
    """Count total and trainable parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Count shared feature matrices separately
    registry = GlobalFeature()
    shared_params = 0
    for key, value in registry._feature_matrices.items():
        if isinstance(value, nn.Parameter):
            shared_params += value.numel()
        elif isinstance(value, dict):
            for v in value.values():
                if isinstance(v, nn.Parameter):
                    shared_params += v.numel()

    return {
        'total': total,
        'trainable': trainable,
        'shared_features': shared_params
    }


def compare_models():
    """Compare standard GPT-2 vs Shared Tversky GPT-2."""
    print("\n" + "="*70)
    print("MODEL COMPARISON: Standard vs Shared Tversky")
    print("="*70)

    # Small config for demonstration
    config = GPT2Config(
        vocab_size=50257,
        n_positions=1024,
        n_embd=768,
        n_layer=12,
        n_head=12,
    )

    # Standard GPT-2 (reference)
    from transformers import GPT2LMHeadModel
    standard_model = GPT2LMHeadModel(config)
    standard_params = count_parameters(standard_model)

    print(f"\n1. STANDARD GPT-2")
    print(f"   Total parameters: {standard_params['total']:,}")

    # Shared Tversky GPT-2
    print(f"\n2. BUILDING SHARED TVERSKY GPT-2...")
    shared_model = GPT2SharedTverskyLMHead(config, feature_key='gpt2_shared')
    shared_params = count_parameters(shared_model)

    print(f"\n   SHARED TVERSKY GPT-2")
    print(f"   Total parameters: {shared_params['total']:,}")
    print(f"   Shared feature parameters: {shared_params['shared_features']:,}")
    print(f"   Effective parameters: {shared_params['total'] + shared_params['shared_features']:,}")

    # Calculate reduction
    reduction = (1 - (shared_params['total'] + shared_params['shared_features']) / standard_params['total']) * 100

    print(f"\n3. COMPARISON")
    print(f"   Parameter reduction: {reduction:.1f}%")
    print(f"   Ratio: {(shared_params['total'] + shared_params['shared_features']) / standard_params['total']:.2f}x")

    # Show shared feature matrix count
    registry = GlobalFeature()
    print(f"\n4. SHARED FEATURE MATRICES")
    print(f"   Number of unique feature matrices: {len([k for k in registry._feature_matrices.keys() if 'gpt2_shared' in k and not 'params' in k])}")
    for key in registry._feature_matrices.keys():
        if 'gpt2_shared' in key and not 'params' in key:
            matrix = registry.get_feature(key)
            if isinstance(matrix, nn.Parameter):
                print(f"   - {key}: {matrix.shape}")


# ============================================================================
# Usage Examples
# ============================================================================

def example_basic_shared():
    """Example 1: Basic shared feature usage."""
    print("\n" + "="*70)
    print("Example 1: Basic Shared Feature Matrix")
    print("="*70)

    # Clear registry
    GlobalFeature().clear()

    # Create multiple layers that share features
    layer1 = SharedTverskyLinear(512, 256, feature_key='example')
    layer2 = SharedTverskyLinear(512, 128, feature_key='example')
    layer3 = SharedTverskyLinear(512, 64, feature_key='example')

    # Count parameters
    total_params = (
        sum(p.numel() for p in layer1.parameters()) +
        sum(p.numel() for p in layer2.parameters()) +
        sum(p.numel() for p in layer3.parameters())
    )

    # Count shared features
    registry = GlobalFeature()
    shared_feature_params = registry.get_feature('example_512').numel()

    print(f"\nLayer parameters (prototypes + bias): {total_params:,}")
    print(f"Shared feature matrix parameters: {shared_feature_params:,}")
    print(f"Total effective parameters: {total_params + shared_feature_params:,}")

    # Compare with non-shared
    non_shared_params = 3 * (512 * 512)  # Each layer would have its own feature matrix
    print(f"\nIf features were NOT shared: {total_params + non_shared_params:,}")
    print(f"Savings: {non_shared_params - shared_feature_params:,} parameters")
    print(f"Reduction: {((non_shared_params - shared_feature_params) / (total_params + non_shared_params)) * 100:.1f}%")


def example_full_model():
    """Example 2: Full model with generation."""
    print("\n" + "="*70)
    print("Example 2: Full Model with Text Generation")
    print("="*70)

    config = GPT2Config(
        vocab_size=50257,
        n_positions=512,
        n_embd=384,
        n_layer=6,
        n_head=6,
    )

    model = GPT2SharedTverskyLMHead(config, feature_key='demo')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    # Test generation
    prompt = "Once upon a time"
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    print(f"Prompt: '{prompt}'")
    print("Generating...")

    generated = model.generate(input_ids, max_length=30, temperature=1.0)
    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)

    print(f"Generated: '{generated_text}'")

    # Show parameters
    params = count_parameters(model)
    print(f"\nModel parameters: {params['total']:,}")
    print(f"Shared features: {params['shared_features']:,}")


def example_training():
    """Example 3: Training loop."""
    print("\n" + "="*70)
    print("Example 3: Training with Shared Features")
    print("="*70)

    config = GPT2Config(
        vocab_size=50257,
        n_positions=128,
        n_embd=256,
        n_layer=4,
        n_head=4,
    )

    model = GPT2SharedTverskyLMHead(config, feature_key='training')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    # Training data
    texts = [
        "The quick brown fox jumps over the lazy dog.",
        "Machine learning is transforming technology.",
        "Python is a versatile programming language.",
    ]

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()

    print("Training for 5 steps...")
    for step in range(5):
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()

        # Check that shared features receive gradients
        registry = GlobalFeature()
        feature_matrix = registry.get_feature('training_256')
        has_grad = feature_matrix.grad is not None

        optimizer.step()

        print(f"Step {step+1}: Loss={loss.item():.4f}, Shared features updated={has_grad}")

    print("✓ Training completed!")
    print("✓ Shared feature matrices are being updated during training!")


# ============================================================================
# Main
# ============================================================================

def main():
    print("\n" + "="*70)
    print("GPT-2 WITH GLOBALLY SHARED TVERSKY FEATURE MATRICES")
    print("="*70)

    example_basic_shared()
    compare_models()
    example_full_model()
    example_training()

    print("\n" + "="*70)
    print("KEY INSIGHTS")
    print("="*70)
    print("""
    ✓ Feature matrices are shared across ALL Tversky layers
    ✓ Each layer has its own prototypes (like weights in nn.Linear)
    ✓ Sharing features dramatically reduces parameter count
    ✓ This is how the paper achieves 34.8% parameter reduction
    ✓ Shared features are trainable and receive gradients
    ✓ GlobalFeatureRegistry manages all shared parameters

    Paper Reference: arxiv.org/abs/2506.11035
    """)


if __name__ == "__main__":
    main()


GPT-2 WITH GLOBALLY SHARED TVERSKY FEATURE MATRICES

Example 1: Basic Shared Feature Matrix

Layer parameters (prototypes + bias): 229,824
Shared feature matrix parameters: 262,144
Total effective parameters: 491,968

If features were NOT shared: 1,016,256
Savings: 524,288 parameters
Reduction: 51.6%

MODEL COMPARISON: Standard vs Shared Tversky

1. STANDARD GPT-2
   Total parameters: 124,439,808

2. BUILDING SHARED TVERSKY GPT-2...

Building GPT-2 with Shared Tversky Layers


   SHARED TVERSKY GPT-2
   Total parameters: 163,037,184
   Shared feature parameters: 10,027,014
   Effective parameters: 173,064,198

3. COMPARISON
   Parameter reduction: -39.1%
   Ratio: 1.39x

4. SHARED FEATURE MATRICES
   Number of unique feature matrices: 2
   - gpt2_shared_768: torch.Size([768, 768])
   - gpt2_shared_intermediate_3072: torch.Size([3072, 3072])

Example 2: Full Model with Text Generation

Building GPT-2 with Shared Tversky Layers

Prompt: 'Once upon a time'
Generating...


KeyboardInterrupt: 